# transformers fine tune qa for model: distilbert-base-uncased with dataset squad_v2/squad


In [49]:
import os
import random
import torch
from datasets import load_dataset
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForQuestionAnswering,
    Trainer,
    TrainingArguments,
    default_data_collator,
    pipeline
)
from transformers import DataCollatorWithPadding
from evaluate import load  # Hugging Face evaluate library

from datetime import datetime
from tqdm.auto import tqdm
import collections
import numpy as np

# Set to False for quick testing with smaller subset
use_full_dataset = False

# Create a unique timestamped base output directory
ts = datetime.now().strftime("%Y-%m-%d_%H%M%S")
output_dir_base = f"./finetune/{ts}"
print(f"📂 All outputs will be saved under: {output_dir_base}")

# -------------------------------
# 1. Config: Choose SQuAD or SQuAD 2.0
# -------------------------------
squad_v2 = True  # Set False to use SQuAD 1.1
dataset_name = "squad_v2" if squad_v2 else "squad"
print(f"📚 Loading dataset: {dataset_name}")

datasets_ori = load_dataset(dataset_name)

# -------------------------------
# 2. Load base Tokenizer & Model
# -------------------------------
model_name_base = "distilbert-base-uncased"
model_base = DistilBertForQuestionAnswering.from_pretrained(model_name_base)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name_base)
tokenizer_base = DistilBertTokenizerFast.from_pretrained(model_name_base)

pad_on_right = tokenizer.padding_side == "right"
# The maximum length of a feature (question and context)
max_length = 384 
# The authorized overlap between two part of the context when splitting it is needed.
doc_stride = 128

# -------------------------------
# 3. preprocess the train dataset
# -------------------------------
def prepare_train_features(examples):
    # 一些问题的左侧可能有很多空白字符，这对我们没有用，而且会导致上下文的截断失败
    # （标记化的问题将占用大量空间）。因此，我们删除左侧的空白字符。
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # 使用截断和填充对我们的示例进行标记化，但保留溢出部分，使用步幅（stride）。
    # 当上下文很长时，这会导致一个示例可能提供多个特征，其中每个特征的上下文都与前一个特征的上下文有一些重叠。
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # 由于一个示例可能给我们提供多个特征（如果它具有很长的上下文），我们需要一个从特征到其对应示例的映射。这个键就提供了这个映射关系。
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # 偏移映射将为我们提供从令牌到原始上下文中的字符位置的映射。这将帮助我们计算开始位置和结束位置。
    offset_mapping = tokenized_examples.pop("offset_mapping")					  

    # 让我们为这些示例进行标记！
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # 我们将使用 CLS 特殊 token 的索引来标记不可能的答案。
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # 获取与该示例对应的序列（以了解上下文和问题是什么）。
        sequence_ids = tokenized_examples.sequence_ids(i)

        # 一个示例可以提供多个跨度，这是包含此文本跨度的示例的索引。
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # 如果没有给出答案，则将cls_index设置为答案。
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # 答案在文本中的开始和结束字符索引。
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # 当前跨度在文本中的开始令牌索引。
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # 当前跨度在文本中的结束令牌索引。
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # 检测答案是否超出跨度（在这种情况下，该特征的标签将使用CLS索引）。
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # 否则，将token_start_index和token_end_index移到答案的两端。
                # 注意：如果答案是最后一个单词（边缘情况），我们可以在最后一个偏移之后继续。
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples
										   

encoded_datasets = datasets_ori.map(
    prepare_train_features,
    batched=True,
    remove_columns=datasets_ori["train"].column_names
)

# define the train, eval dataset
if use_full_dataset:
    print("Using full dataset for training and evaluation.")
    train_dataset=encoded_datasets["train"]
    eval_dataset=encoded_datasets["validation"]
    eval_dataset_ori = datasets_ori["validation"]
else:
    print("Using smaller subset of 1000 examples for quick testing.")
    train_dataset=encoded_datasets["train"].shuffle(seed=42).select(range(100))
    eval_dataset=encoded_datasets["validation"].shuffle(seed=42).select(range(10))
    eval_dataset_ori = datasets_ori["validation"].select(range(10))  # small subset for speed



# -------------------------------
# 4. Training & Saving Models (First & Second Fine-Tuning)
# -------------------------------
output_dirs = {
    "base": f"{output_dir_base}/base",
    "first": f"{output_dir_base}/first",
    "second": f"{output_dir_base}/second"
}

for key in output_dirs:
    os.makedirs(output_dirs[key], exist_ok=True)

def fine_tune_and_save(
    pass_name,
    output_dir,
    train_epochs=3,
    lr=3e-5,
    batch_size=16,
    model_path=None  # Optional: path to a previously saved model (e.g., first fine-tuned)
):
    print(f"🔁 Starting {pass_name} fine-tuning pass...")
    
    # Load model & tokenizer
    if model_path:
        print(f"📥 Loading model & tokenizer from: {model_path}")
        model = DistilBertForQuestionAnswering.from_pretrained(model_path)
        tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
    else:
        print(f"📥 Loading base model: distilbert-base-uncased")
        model = model_base
        tokenizer = tokenizer_base

    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"{output_dir}/results",
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=train_epochs,
        weight_decay=0.01,
        logging_dir=f"{output_dir}/logs",
        logging_steps=1,
        save_total_limit=3,
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
    )

    # Train
    trainer.train()
    
    # Save model & tokenizer
    saved_model_dir = f"{output_dir}/models/{model_name_base}-finetuned-{pass_name}"
    trainer.save_model(saved_model_dir)
    tokenizer.save_pretrained(saved_model_dir)
    print(f"✅ {pass_name.capitalize()} model saved to: {saved_model_dir}")

    return saved_model_dir

# Fine-tune first model from base
first_model_dir = fine_tune_and_save(
    pass_name="first",
    output_dir=output_dirs["first"],
    train_epochs=2,
    lr=3e-5,
    model_path=None  # Starts from base
)
# Fine-tune second model using the first fine-tuned model
second_model_dir = fine_tune_and_save(
    pass_name="second",
    output_dir=output_dirs["second"],
    train_epochs=1,
    lr=2e-5,
    model_path=first_model_dir  # Loads first fine-tuned model
)

# -------------------------------
# 5. EVALUATION: Exact Match (EM) & F1 using Hugging Face `evaluate`
# -------------------------------

squad_metric = load("squad_v2" if squad_v2 else "squad")  # ⚠️ important: use same as dataset!

def prepare_validation_features(examples):
    # 一些问题的左侧有很多空白，这些空白并不有用且会导致上下文截断失败（分词后的问题会占用很多空间）。
    # 因此我们移除这些左侧空白
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # 使用截断和可能的填充对我们的示例进行分词，但使用步长保留溢出的令牌。这导致一个长上下文的示例可能产生
    # 几个特征，每个特征的上下文都会稍微与前一个特征的上下文重叠。
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # 由于一个示例在上下文很长时可能会产生几个特征，我们需要一个从特征映射到其对应示例的映射。这个键就是为了这个目的。
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # 我们保留产生这个特征的示例ID，并且会存储偏移映射。
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # 获取与该示例对应的序列（以了解哪些是上下文，哪些是问题）。
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # 一个示例可以产生几个文本段，这里是包含该文本段的示例的索引。
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # 将不属于上下文的偏移映射设置为None，以便容易确定一个令牌位置是否属于上下文。
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # 构建一个从示例到其对应特征的映射。
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # 我们需要填充的字典。
    predictions = collections.OrderedDict()

    # 日志记录。
    print(f"正在后处理 {len(examples)} 个示例的预测，这些预测分散在 {len(features)} 个特征中。")

    # 遍历所有示例！
    for example_index, example in enumerate(tqdm(examples)):
        # 这些是与当前示例关联的特征的索引。
        feature_indices = features_per_example[example_index]

        min_null_score = None # 仅在squad_v2为True时使用。
        valid_answers = []
        
        context = example["context"]
        # 遍历与当前示例关联的所有特征。
        for feature_index in feature_indices:
            # 我们获取模型对这个特征的预测。
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # 这将允许我们将logits中的某些位置映射到原始上下文中的文本跨度。
            offset_mapping = features[feature_index]["offset_mapping"]

            # 更新最小空预测。
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # 浏览所有的最佳开始和结束logits，为 `n_best_size` 个最佳选择。
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # 不考虑超出范围的答案，原因是索引超出范围或对应于输入ID的部分不在上下文中。
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # 不考虑长度小于0或大于max_answer_length的答案。
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            sorted_valid_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)
            print(f"sorted valid_ansers={sorted_valid_answer}")
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # 在极少数情况下我们没有一个非空预测，我们创建一个假预测以避免失败。
            best_answer = {"text": "", "score": 0.0}
        
        # 选择我们的最终答案：最佳答案或空答案（仅适用于squad_v2）
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

def evaluate_model_squad_format(model_dir, output_dir):
    print(f"🔍 Evaluating model: {model_dir}")
    model = DistilBertForQuestionAnswering.from_pretrained(model_dir)
    tokenizer = DistilBertTokenizerFast.from_pretrained(model_dir)
    
    validation_features = eval_dataset_ori.map(
        prepare_validation_features,
        batched=True,
        remove_columns=eval_dataset_ori.column_names
    )

    training_args = TrainingArguments(
        output_dir=f"{output_dir}/results",
        per_device_eval_batch_size=8,
        do_train=False,          # we're not training
        do_eval=False,           # not evaluating either
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
    )

    raw_predictions = trainer.predict(validation_features)
    validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

    final_predictions = postprocess_qa_predictions(eval_dataset_ori, validation_features, raw_predictions.predictions)

    if squad_v2:
        formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
    else:
        formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
    
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in eval_dataset_ori]

    results = squad_metric.compute(predictions=formatted_predictions, references=references)
    print(f"results={results}")
    print(f"📊 Exact Match (EM): {results['exact']:.2f} | F1: {results['f1']:.2f}")


# Evaluate all 3 models
evaluate_model_squad_format(model_name_base, output_dirs["base"])  # Base model
evaluate_model_squad_format(first_model_dir, output_dirs["first"])   # First fine-tuned
evaluate_model_squad_format(second_model_dir, output_dirs["second"])  # Second fine-tuned

# -------------------------------
# 6. (Optional) Sample Q/A Predictions (Qualitative)
# -------------------------------
def test_model_predictions(model_dir, samples):
    print(f"--- Sample QA from Model: {model_dir} ---")
    model = DistilBertForQuestionAnswering.from_pretrained(model_dir)
    tokenizer = DistilBertTokenizerFast.from_pretrained(model_dir)
    qa_pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device="cuda" if torch.cuda.is_available() else -1)

    for i, example in enumerate(samples):  # Correct: loop over rows directly
        question = example["question"]
        context = example["context"]
        ta = example["answers"]["text"] # Ground truth answer
        result = qa_pipe(question=question, context=context)
        print(f"Q{i + 1}: {question}")
        print(f"C{i + 1}: {context[:200]}...")
        print(f"A{i + 1}: {result['answer']}")
        print(f"GTA{i + 1}: {ta[:100]}\n")  # Show first 100 chars of ground truth answer

print("🧪 Running Sample QA Predictions...")
samples = datasets_ori["validation"].select(range(3))  # Get first 3 examples
test_model_predictions(model_name_base, samples)   # Base
test_model_predictions(first_model_dir, samples)    # First fine-tuned
test_model_predictions(second_model_dir, samples)   # Second fine-tuned

📂 All outputs will be saved under: ./finetune/2025-08-01_113558
📚 Loading dataset: squad_v2


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using smaller subset of 1000 examples for quick testing.
🔁 Starting first fine-tuning pass...
📥 Loading base model: distilbert-base-uncased


C:\Users\bill\AppData\Local\Temp\ipykernel_1652\3676611870.py:197: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
C:\dev\inst\miniconda3\envs\py311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,5.764800,5.602178
2,5.453100,5.398049


...

Q1: In what country is Normandy located?
C1: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("...
A1: Carolingian-
GTA1: ['France', 'France', 'France', 'France']

Q2: When were the Normans in Normandy?
C2: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("...
A2: Carolingian-
GTA2: ['10th and 11th centuries', 'in the 10th and 11th centuries', '10th and 11th centuries', '10th and 11th centuries']



Device set to use cpu


Q3: From which countries did the Norse originate?
C3: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("...
A3: Carolingian-
GTA3: ['Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway']

--- Sample QA from Model: ./finetune/2025-08-01_113558/first/models/distilbert-base-uncased-finetuned-first ---
Q1: In what country is Normandy located?
C1: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("...
A1: half of
GTA1: ['France', 'France', 'France', 'France']

Q2: When were the Normans in Normandy?
C2: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their 

Device set to use cpu


Q3: From which countries did the Norse originate?
C3: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("...
A3: half of
GTA3: ['Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway']

--- Sample QA from Model: ./finetune/2025-08-01_113558/second/models/distilbert-base-uncased-finetuned-second ---
Q1: In what country is Normandy located?
C1: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("...
A1: 10th
GTA1: ['France', 'France', 'France', 'France']

Q2: When were the Normans in Normandy?
C2: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name t